# CBAM 深度解析与 LSTM 融合实战手册

## 1. CBAM 是什么？（核心直觉）

**CBAM (Convolutional Block Attention Module)** 是一种轻量级的注意力机制模块。

你可以把它想象成人类视觉的处理过程：当我们看一张图时，我们不会盯着每个像素看，而是会遵循以下逻辑：
1.  **“这是什么”**（比如这是一只猫）
2.  **“在哪儿”**（猫在草地上）

CBAM 正是通过两个步骤来模仿这个过程：

* **通道注意力 (Channel Attention)**：关注 **"What"**（什么特征最重要？是纹理？是形状？）。
* **空间注意力 (Spatial Attention)**：关注 **"Where"**（重要信息在图像/序列的哪个位置？）。

---

## 2. 核心原理与公式拆解

CBAM 接收一个特征图 $\mathbf{F}$ 作为输入，依次经过 **通道注意力模块** 和 **空间注意力模块**，最终输出精炼后的特征。

### 2.1 通道注意力模块 (Channel Attention Module, CAM)

* **目的**：给不同的通道（Channel）打分。在 LSTM 语境下，就是给 Hidden State 的不同维度打分。
* **核心机制**：
    > 这就好比大家在开会，为了总结会议精神（提取特征），我们既需要听取“平均意见”（平均池化），也需要听取“最尖锐的意见”（最大池化）。

#### 公式推导

$$
\begin{aligned}
\mathbf{M_c}(\mathbf{F}) &= \sigma(MLP(AvgPool(\mathbf{F})) + MLP(MaxPool(\mathbf{F}))) \\
&= \sigma(\mathbf{W_1}(\mathbf{W_0}(\mathbf{F}_{avg}^c)) + \mathbf{W_1}(\mathbf{W_0}(\mathbf{F}_{max}^c)))
\end{aligned}
$$

#### 详细解释

1.  **$AvgPool(\mathbf{F})$ 和 $MaxPool(\mathbf{F})$**：
    * **操作**：分别对空间维度（或时间维度）进行全局平均池化和全局最大池化。
    * **意义**：压缩空间信息，只保留通道维度的“指纹”。
2.  **$MLP$ (多层感知机)**：
    * **操作**：一个共享权重的两层神经网络（先降维再升维）。
    * **意义**：捕捉通道之间的非线性依赖关系。
3.  **相加 (+)**：
    * 将两组特征融合，使特征描述更鲁棒。
4.  **$\sigma$ (Sigmoid 函数)**：
    * **公式**：$\frac{1}{1+e^{-x}}$
    * **意义**：将输出压缩到 $0$ 到 $1$ 之间，生成“权重系数”。

---

### 2.2 空间注意力模块 (Spatial Attention Module, SAM)

* **目的**：给空间（或时间序列）上的不同位置打分。在 LSTM 语境下，就是判断哪个时间步（Time Step）最重要。

#### 公式推导

$$
\mathbf{M_s}(\mathbf{F}) = \sigma(f^{7\times7}([\mathbf{F}^s_{avg}; \mathbf{F}^s_{max}]))
$$

#### 详细解释

1.  **$[\mathbf{F}^s_{avg}; \mathbf{F}^s_{max}]$**：
    * **操作**：在通道维度上进行平均池化和最大池化，然后拼接 (Concat) 在一起。
    * **结果**：将厚厚的通道压缩成 2 层（一层平均，一层最大）。就像把一本书压扁成一张纸，只保留每一页在该位置的概要信息。
2.  **$f^{7\times7}$ (卷积层)**：
    * **操作**：使用一个大卷积核（通常是 $7\times7$ 或 $3\times3$）进行卷积。
    * **意义**：融合局部信息，判断哪些区域（位置）是“热点”。
3.  **$\sigma$ (Sigmoid)**：
    * 生成空间（时间）位置的权重图。

---

## 3. CBAM 与 LSTM 的融合策略

### 维度差异
* **标准 CBAM**：通常处理 4D 张量 $(N, C, H, W)$（用于图像）。
* **LSTM 输出**：通常是 3D 张量 $(N, L, H_{in})$，其中：
    * $N$: Batch Size
    * $L$: Sequence Length (时间步长)
    * $H_{in}$: Hidden Size (特征维度)

### 融合方案
**解决方案**：我们将 1D-CBAM 应用于 LSTM 的输出。我们将 $L$ 看作空间维度，$H_{in}$ 看作通道维度。

1.  **通道注意力 (Channel Attention)**：
    * **作用对象**：$H_{in}$
    * **含义**：判断在当前的隐状态中，哪些特征维度（Feature Dimensions）更重要。

2.  **时间注意力 (原空间注意力, Spatial Attention)**：
    * **作用对象**：$L$
    * **含义**：判断在整个序列中，哪些时间点（Time Steps）包含关键信息。

# 1D CBAM

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
# 我们统一约定输入张量的维度表示为 [N, C, L]
# N (Batch Size)：批次大小。
# C (Channels)：通道数。在 LSTM 结合场景下，这对应 LSTM 的 Hidden_Size（特征维度）。
# L (Length/Sequence Length)：序列长度。在 LSTM 场景下，这对应时间步长（Time Steps）

class ChannelAttention1D(nn.Module):
    """
    针对 1D 序列的通道注意力 (Channel Attention)
    核心目的：找出哪个“特征通道 (C)”最重要，忽略不重要的特征。
    """
    def __init__(self, in_planes, ratio=16):
        # 语法: 初始化父类 nn.Module
        super().__init__()
        # in_planes是输入的通道数
        # --- 1. 空间(时间)池化层 ---
        # 语法: AdaptiveAvgPool1d(1) 自适应一维平均池化，参数 1 表示输出的目标长度(L)为 1。
        # 作用: 将整个时间序列的信息“浓缩”成一个平均值。
        # 预期维度变化: [N, C, L] -> [N, C, 1]
        self.avg_pool = nn.AdaptiveAvgPool1d(1)
        
        # 语法: AdaptiveMaxPool1d(1) 自适应一维最大池化，输出目标长度为 1。
        # 作用: 提取整个时间序列中最显著的、最强烈的信号特征。
        # 预期维度变化: [N, C, L] -> [N, C, 1]
        self.max_pool = nn.AdaptiveMaxPool1d(1)
        
        # --- 2. 共享多层感知机 (Shared MLP) ---
        # 语法: nn.Sequential 按顺序组合多个网络层。这里用 1D 卷积来模拟全连接层 (Linear)。
        # 作用: 让通道之间互相交换信息，学习通道之间的非线性依赖关系。
        self.sharedMLP = nn.Sequential(
            # 第一层卷积 (降维)
            # 语法: nn.Conv1d(输入通道, 输出通道, 卷积核大小)。这里核为1，相当于对每个时间点独立做线性映射。
            # 参数: in_planes // ratio 将通道数压缩 16 倍，减少计算量并提取核心特征。
            # 预期维度变化: [N, C, 1] -> [N, C/16, 1]
            nn.Conv1d(in_planes, in_planes // ratio, 1, bias=False),
            
            # 激活函数
            # 语法: ReLU 激活函数，增加非线性。维度不改变。
            nn.ReLU(),
            
            # 第二层卷积 (升维)
            # 语法: 再次使用 1x1 卷积，将通道数恢复到原始大小。
            # 预期维度变化: [N, C/16, 1] -> [N, C, 1]
            nn.Conv1d(in_planes // ratio, in_planes, 1, bias=False)
        )
        
        # 语法: Sigmoid 函数，将输入映射到 (0, 1) 区间。
        # 作用: 生成最终的通道注意力权重系数。
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # 参数 x 的初始维度: [N, C, L]
        
        # 语法: self.avg_pool(x) 先平均池化，再送入 self.sharedMLP 计算。
        # 维度流转: [N, C, L] -> (经过池化) -> [N, C, 1] -> (经过MLP降维升维) -> [N, C, 1]
        avg_out = self.sharedMLP(self.avg_pool(x))
        
        # 语法: self.max_pool(x) 先最大池化，再送入同一个 self.sharedMLP。
        # 维度流转: [N, C, L] -> (经过池化) -> [N, C, 1] -> (经过MLP降维升维) -> [N, C, 1]
        max_out = self.sharedMLP(self.max_pool(x))
        
        # 语法: 两个张量在对应元素上直接相加 (Element-wise addition)。
        # 作用: 融合平均特征和最大特征，使通道权重评估更全面。
        # 维度变化: [N, C, 1] + [N, C, 1] -> 输出 out 维度仍为 [N, C, 1]
        out = avg_out + max_out
        
        # 语法 & 作用: 经过 Sigmoid 激活，输出 (0~1) 的权重。
        # 最终输出维度: [N, C, 1]
        return self.sigmoid(out)


class SpatialAttention1D(nn.Module):
    """
    针对 1D 序列的空间(时间)注意力 (Spatial/Temporal Attention)
    核心目的：找出哪个“时间步 (L)”最重要，忽略无关的时间段。
    """
    def __init__(self, kernel_size=7):
        super().__init__()
        # 语法: 断言检查，确保卷积核大小只能是 3 或 7 (经验值)。
        assert kernel_size in (3, 7), 'kernel size must be 3 or 7'
        
        # 语法: 根据 kernel_size 动态设置 padding (填充大小)，以保证卷积操作前后序列长度 L 不变。
        # 规则: padding = (kernel_size - 1) / 2
        padding = 3 if kernel_size == 7 else 1
        
        # 语法: 定义 1D 卷积层。输入通道为 2，输出通道为 1。
        # 参数作用: 
        #   in_channels=2: 接收后面在通道维度上拼接的 avg_out 和 max_out。
        #   out_channels=1: 最终只需输出一个综合的时间步权重。
        self.conv1 = nn.Conv1d(2, 1, kernel_size, padding=padding, bias=False)
        
        # 语法: Sigmoid 函数，将输出映射为 (0, 1) 的权重。
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # 参数 x 的初始维度: [N, C, L] (注意：这里的输入通常是已经过通道注意力加权后的特征)
        
        # 语法: torch.mean 计算平均值; dim=1 指定在“通道 C”这个维度上操作; keepdim=True 保持维度结构。
        # 作用: 提取每个时间步在所有通道上的平均表现。把厚厚的 C 压扁。
        # 维度变化: [N, C, L] -> [N, 1, L]
        avg_out = torch.mean(x, dim=1, keepdim=True) 
        
        # 语法: torch.max 计算最大值; dim=1 在通道维度操作; 返回值是一个元组 (max_values, max_indices)，我们用 _ 丢弃索引。
        # 作用: 提取每个时间步在所有通道上的最强表现。
        # 维度变化: [N, C, L] -> [N, 1, L]
        max_out, _ = torch.max(x, dim=1, keepdim=True) 
        
        # 语法: torch.cat 进行拼接; dim=1 指定在通道维度上进行拼接。
        # 作用: 将平均图和最大图叠在一起，交给卷积层去评判。
        # 维度变化: [N, 1, L] 拼接 [N, 1, L] -> 结果 x 维度变为 [N, 2, L]
        x = torch.cat([avg_out, max_out], dim=1) 
        
        # 语法: 经过前面定义的 1D 卷积层。
        # 作用: 跨越时间步(基于 kernel_size 感受野)融合局部时间信息，计算当前时间步的重要性。
        # 维度变化: [N, 2, L] -> (卷积) -> [N, 1, L]
        x = self.conv1(x) 
        
        # 语法 & 作用: 经过 Sigmoid 激活，输出时间步的注意力权重。
        # 最终输出维度: [N, 1, L]
        return self.sigmoid(x)


class CBAM1D(nn.Module):
    """
    完整的 1D CBAM 模块
    将通道注意力和空间(时间)注意力串联起来
    """
    def __init__(self, in_planes, ratio=16, kernel_size=7):
        super().__init__()
        # 实例化通道注意力模块 (传入特征通道数 C 和降维比例)
        self.ca = ChannelAttention1D(in_planes, ratio)
        # 实例化空间(时间)注意力模块 (传入卷积核大小)
        self.sa = SpatialAttention1D(kernel_size)

    def forward(self, x):
        # 参数 x 初始维度: [N, C, L] 
        # (重要提醒: LSTM 原本输出是 [N, L, C]，进入这里之前必须要经过 x.permute(0, 2, 1) 转换)
        
        # --- 1. 通道注意力加权 ---
        # 语法: self.ca(x) 计算得到通道权重，维度为 [N, C, 1]。
        # 语法: x * self.ca(x) 是按元素相乘。PyTorch 会自动触发“广播机制 (Broadcasting)”。
        # 广播过程: [N, C, 1] 自动在长度 L 维度上复制扩充成 [N, C, L]，然后与 x [N, C, L] 逐元素相乘。
        # 作用: 根据权重放缩每一个通道的值 (重要的通道数值放大，不重要的缩小)。
        # 维度变化: [N, C, L] * [N, C, 1] -> 输出 out 维度为 [N, C, L]
        out = x * self.ca(x)
        
        # --- 2. 空间(时间)注意力加权 ---
        # 语法: self.sa(out) 接收通道加权后的特征，计算时间步权重，维度为 [N, 1, L]。
        # 语法: out * self.sa(out) 同样按元素相乘，触发广播机制。
        # 广播过程: [N, 1, L] 自动在通道 C 维度上复制扩充成 [N, C, L]，然后与 out [N, C, L] 逐元素相乘。
        # 作用: 根据权重放缩每一个时间步的值 (重要的高亮，噪音抑制)。
        # 维度变化: [N, C, L] * [N, 1, L] -> 最终结果 result 维度为 [N, C, L]
        result = out * self.sa(out)
        
        # 最终返回经过两次精炼的特征张量
        return result

# LSTM + CBAM

In [3]:
import torch
import torch.nn as nn
# $N$: Batch Size (批次大小)
# $L$: Seq Len (时间序列长度)
# $C_{in}$: Input Size (输入特征维度)
# $C_{out}$: Hidden Size (LSTM 的隐藏层维度，也就是 CBAM 视作的“通道数”)
# $K$: Num Classes (分类数或预测目标维度)

class LSTM_CBAM_Model(nn.Module):
    """
    将 LSTM 与 1D CBAM 结合的复合神经网络模型
    """
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        # 语法: 初始化父类 nn.Module
        # 作用: 注册当前类为一个合法的 PyTorch 模块，使其能够管理网络层和权重参数。
        super().__init__()
        
        # 语法: 将传入的 hidden_size 保存为类的属性。
        # 作用: 方便在类的其他方法（如 forward）中随时调用。
        self.hidden_size = hidden_size
        
        # --- 1. 定义 LSTM 层 ---
        # 语法: 实例化多层 LSTM 网络。
        # 参数:
        #   input_size: 每个时间步输入数据的特征数 (Cin)。
        #   hidden_size: LSTM 内部记忆单元的特征维度 (Cout)。
        #   num_layers: LSTM 的层数（例如 2 表示堆叠了两层 LSTM）。
        #   batch_first=True: 【极度重要】设定输入和输出张量的第一维是 Batch Size。
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        
        # --- 2. 定义 CBAM 模块 ---
        # 语法: 实例化我们之前写好的 1D CBAM 类。
        # 参数: in_planes=hidden_size。
        # 作用: 告诉 CBAM，你要处理的“通道数”等于 LSTM 提取出的特征维度。
        self.cbam = CBAM1D(in_planes=hidden_size)
        
        # --- 3. 定义全连接分类头 ---
        # 语法: 实例化线性层 (Linear/Dense)。
        # 参数: 输入维度为 hidden_size，输出维度为 num_classes。
        # 作用: 将经过 LSTM 提取、CBAM 精炼后的高维特征，映射到最终的预测标签上。
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # 【输入起始点】 x 的初始维度: [N, L, Cin]
        # (例如: [32, 50, 10] -> 32个样本，每个样本50天，每天10个指标)
        
        # --- 1. LSTM 前向传播 ---
        # 语法: 将 x 传入 lstm 层。返回两个值：所有的输出序列 lstm_out，以及最后一个时间步的隐状态 _ (此处用_丢弃)。
        # 作用: 捕捉时间序列的长短期依赖关系。
        # 维度变化: [N, L, Cin]  ->  [N, L, Cout]
        # (例如: [32, 50, 10] -> [32, 50, 64])
        lstm_out, _ = self.lstm(x)
        
        # --- 2. 核心桥梁：维度调整 (Permute) ---
        # 语法: .permute(0, 2, 1) 重新排列张量的维度顺序。
        # 作用: PyTorch 的 LSTM 输出格式是 (Batch, Length, Channels)。
        #       但是，PyTorch 的所有 Conv1d 层（CBAM内部）要求的格式是 (Batch, Channels, Length)。
        #       所以必须把 第2维(L) 和 第3维(Cout) 调换位置！
        # 维度变化: [N, L, Cout]  ->  [N, Cout, L]
        # (例如: [32, 50, 64] -> [32, 64, 50])
        cbam_input = lstm_out.permute(0, 2, 1) 
        
        # --- 3. 应用 CBAM 精炼特征 ---
        # 语法: 将调换维度后的张量送入 CBAM 模块。
        # 作用: 进行通道打分（剔除冗余特征）和时间步打分（剔除噪音时间段）。
        # 维度变化: 输入和输出维度完全一致，保持不变。
        # (例如: [32, 64, 50] -> [32, 64, 50])
        refined_features = self.cbam(cbam_input)
        
        # --- 4. 聚合序列特征 ---
        # 语法: torch.mean 计算平均值，dim=2 表示沿着“序列长度 L”所在的维度进行计算。
        # 作用: 全局平均池化 (Global Average Pooling 1D)。
        #       无论序列原本有多长 (L=50)，我们把它在时间轴上压扁，求出一个代表整个序列的综合特征向量。
        #       因为前面 CBAM 的 Spatial Attention 已经给重要时间步赋予了大权重，
        #       所以这里的“平均”实际上是被注意力机制优化过的“加权平均”。
        # 维度变化: [N, Cout, L]  ->  [N, Cout]
        # (例如: [32, 64, 50] -> [32, 64])
        final_feature = torch.mean(refined_features, dim=2) 
        
        # --- 5. 全连接层预测 ---
        # 语法: 将压扁后的 2D 张量送入线性分类层。
        # 作用: 输出最终的预测概率分布或回归数值。
        # 维度变化: [N, Cout]  ->  [N, K]
        # (例如: [32, 64] -> [32, 2])
        prediction = self.fc(final_feature)
        
        return prediction

# ==========================================
# --- 模拟运行与测试模块 ---
# ==========================================
# 语法: 确保这部分代码只有在直接运行该脚本时才会执行，被其他文件 import 时不执行。
if __name__ == "__main__":
    
    # 1. 设定模拟数据的超参数
    BATCH_SIZE = 32    # N: 一次处理 32 条独立的时间序列样本
    SEQ_LEN = 50       # L: 每条序列包含 50 个时间步（如 50 天的数据）
    INPUT_SIZE = 10    # Cin: 每个时间步有 10 维特征（如开盘价、收盘价、成交量等）
    HIDDEN_SIZE = 64   # Cout: LSTM 提取出的高维特征维度，也就是进入 CBAM 的通道数
    NUM_CLASSES = 2    # K: 最终分类目标数（比如预测涨或跌，即二分类）
    
    # 2. 生成随机输入张量 (模拟真实的 DataLoader 输出)
    # 语法: torch.randn 生成符合标准正态分布的随机数张量。
    # 初始维度: [32, 50, 10] 对应 [N, L, Cin]
    input_tensor = torch.randn(BATCH_SIZE, SEQ_LEN, INPUT_SIZE)
    
    # 3. 实例化我们刚才定义的复合模型
    # 传入输入特征数、隐藏层大小、LSTM层数和分类数
    model = LSTM_CBAM_Model(INPUT_SIZE, HIDDEN_SIZE, num_layers=2, num_classes=NUM_CLASSES)
    
    # 4. 前向传播计算
    # 语法: 将张量直接传入模型实例，自动调用 forward 方法。
    output = model(input_tensor)
    
    # 5. 打印维度核对结果
    print(f"输入尺寸: {input_tensor.shape}") # 预期: torch.Size([32, 50, 10])
    print(f"输出尺寸: {output.shape}")       # 预期: torch.Size([32, 2])

输入尺寸: torch.Size([32, 50, 10])
输出尺寸: torch.Size([32, 2])
